<a href="https://colab.research.google.com/github/GenZBoy/Automobile-Sales-database/blob/main/plantdiseasepredictionmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rashikrahmanpritom/plant-disease-recognition-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.25G/1.25G [00:40<00:00, 33.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/rashikrahmanpritom/plant-disease-recognition-dataset/versions/1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
# Hyperparameters
num_epochs = 10
learning_rate = 0.001
batch_size = 32
num_classes = 10  # Example: Number of plant diseases

# Transformations (Resize, Grayscale, Normalize, and Data Augmentation)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # EfficientNet input size
    transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel grayscale
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Pretrained weights mean and std
])


In [5]:
full_dataset = datasets.ImageFolder(root=path, transform=transform)

# Split dataset into train and test sets
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [14]:
class EfficientNet_NeuralNet(nn.Module):
    def __init__(self, num_diseases):
        super(EfficientNet_NeuralNet, self).__init__()
        self.efficientnet = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
        in_features = self.efficientnet.classifier[1].in_features
        self.efficientnet.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Linear(512, num_diseases)
        )

    def forward(self, x):
        return self.efficientnet(x)


In [15]:
# Initialize model
model = EfficientNet_NeuralNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [16]:
# Training loop
def train_model():
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


In [17]:
# Evaluation loop
def evaluate_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total:.2f}%')


In [19]:
if __name__ == "__main__":
    train_model()
    evaluate_model()

Epoch [1/10], Loss: 0.7152
Epoch [2/10], Loss: 0.4988
Epoch [3/10], Loss: 0.4364
Epoch [4/10], Loss: 0.3941
Epoch [5/10], Loss: 0.3544
Epoch [6/10], Loss: 0.3473
Epoch [7/10], Loss: 0.3144
Epoch [8/10], Loss: 0.2390
Epoch [9/10], Loss: 0.2428
Epoch [10/10], Loss: 0.2372
Accuracy: 84.69%
